## 匯入必要套件

In [ ]:
# For drawing
import matplotlib.pyplot as plt
# For data processing
import numpy as np
import os
# For buildind model
import tensorflow as tf
from tensorflow.keras.applications import MobileNet

## 資料處理

In [ ]:
# 匯入資料集
from keras.datasets import cifar10
# 資料集切割成訓練與測試資料
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [ ]:
# 資料大小
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape)
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape)

In [ ]:
# 對資料樣本作類別標籤(OneHot Code)
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)

# 由於MobileNetV2 的影像格式是[-1, 1]，且大小最小是[96, 96]，因此用下式將資料轉換相應格式
x_img_train_normalize = tf.image.resize(x_img_train/127.5 - 1.0, [96, 96])  # [0, 255] => [-1, 1]
x_img_test_normalize = tf.image.resize(x_img_test/127.5 - 1.0, [96, 96])  # [0, 255] => [-1, 1]

### 影像縮放

In [ ]:
# 建立資料擴增函式
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'), # 水平翻轉
  tf.keras.layers.RandomRotation(0.2), # 旋轉
])

## 建立模型

In [ ]:
# 定義遷移式學習的基底模型
base_model = tf.keras.applications.MobileNetV2(input_shape=(96,96,3),
                                               include_top=False, # 不包含頂層分類器
                                               weights='imagenet')

In [ ]:
fine_tune = 100
for layer in base_model.layers[:fine_tune]:
  layer.trainable = False   # 前100層不允許調整

inputs = tf.keras.Input(shape=(96,96,3))
x = base_model(inputs)  # base model
x = tf.keras.layers.GlobalAveragePooling2D()(x) # 5x5 Average Pooling 平均空間
x = tf.keras.layers.Dropout(0.2)(x) # 20% 節點隨機設成 0 輸出
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)  # 換算成機率值
model = tf.keras.Model(inputs, outputs) # 定義新模型的名稱

loss_fn = tf.keras.losses.CategoricalCrossentropy() # 使用稀疏交叉熵作為損失函數
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005) # 用 Adam 優化器 (學習率0.005)

model.compile(optimizer=optimizer,  # 指定優化器
              loss=loss_fn, # 指定損失函數
              metrics=['accuracy']) # 評估指標用「正確率(答對的比率)」

print(model.summary())  # 顯示模型結構

In [ ]:
history = model.fit(x_img_train_normalize, y_label_train_OneHot, batch_size=64, epochs=10, validation_split=0.2)  # 訓練 10 epochs

## 顯示準確率 / 損失率

In [ ]:
# 定義繪製訓練準確率與損失率方法
def show_history(history, train_acc, test_acc, train_loss, test_loss):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # 顯示準確率曲線
    ax1.plot(history.history[train_acc])
    ax1.plot(history.history[test_acc])
    ax1.set_title('Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['train', 'test'], loc='upper left')

    # 顯示損失率曲線
    ax2.plot(history.history[train_loss])
    ax2.plot(history.history[test_loss])
    ax2.set_title('Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['train', 'test'], loc='upper left')

    plt.tight_layout()
    plt.show()

In [ ]:
# 繪製準確率 / 損失率
show_history(history,'accuracy','val_accuracy', 'loss', 'val_loss')

## 測試

In [ ]:
# 測試組的預測機率
prediction = np.argmax(model.predict(x_img_test_normalize), axis=-1)

import pandas as pd
# 10 種類別的標記名稱
label_dict = {0:"airplane", 1:"automobile", 2:"bird", 3:"cat", 4:"deer",
              5:"dog", 6:"frog", 7:"horse", 8:"ship", 9:"truck"}
# 顯示混淆矩陣(測試組)
pd.crosstab(y_label_test.reshape(-1), prediction,
            rownames=['label'], colnames=['predict'])